In [1]:
import numpy as np, pandas as pd, json, glob, os, chromadb, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
import shutil

/home/ubuntu2204/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
import json
import shutil
from tqdm import tqdm
from langchain_ollama import OllamaEmbeddings
import chromadb

# -------------------------
# 1️⃣ DB 초기화
# -------------------------
db_path = "./chroma_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)
    print("기존 ChromaDB 삭제 완료")

# -------------------------
# 2️⃣ PersistentClient 연결
# -------------------------
client = chromadb.PersistentClient(path=db_path)

# -------------------------
# 3️⃣ Ollama 임베딩 객체 생성
# -------------------------
ollama_embeddings = OllamaEmbeddings(model="bge-m3")

# -------------------------
# 4️⃣ Collection 생성 (embedding_function 사용)
# -------------------------
collection_name = "medical_qa"
collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=ollama_embeddings.embed_query,  # 임베딩 함수 지정
    metadata={"description": "Medical QA"}
)
print(f"Collection '{collection_name}' 생성 완료")

# -------------------------
# 5️⃣ JSON 폴더 전체 처리 (documents + metadata만 먼저 저장)
# -------------------------
json_root = "../../../../data/medical_data/"  # JSON 상위 폴더

for domain_dir in os.listdir(json_root):
    domain_path = os.path.join(json_root, domain_dir)
    if not os.path.isdir(domain_path):
        continue

    print(f"Processing domain folder: {domain_dir}")

    for file_name in tqdm(os.listdir(domain_path)[:3]):
        file_path = os.path.join(domain_path, file_name)

        # JSON 읽기
        try:
            with open(file_path, "r", encoding="utf-8-sig") as f:
                data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Skipped {file_name}: JSONDecodeError -> {e}")
            continue

        question = data.get("question", "").strip()
        answer = data.get("answer", "").strip()

        if not question:
            print(f"Skipped {file_name}: empty question")
            continue

        metadata = {
            "answer": answer,
            "qa_id": data.get("qa_id"),
            "domain": data.get("domain"),
            "q_type": data.get("q_type")
        }

        # documents + metadata만 먼저 add
        collection.add(
            ids=[file_name],
            documents=[question],
            metadatas=[metadata]
        )

print("모든 JSON 파일의 documents와 metadata 저장 완료!")

# -------------------------
# 6️⃣ 저장 확인
# -------------------------
all_docs = collection.get()
print("총 문서 수:", len(all_docs["ids"]))
print("상위 3개 ID:", all_docs["ids"][:3])
print("상위 3개 question:", all_docs["documents"][:3])
print("상위 3개 metadata:", all_docs["metadatas"][:3])


AttributeError: 'function' object has no attribute 'name'

AttributeError: 'list' object has no attribute 'astype'

In [2]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [3]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                # self.base_url = kargs.get("base_url", "http://localhost:11434")
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            # base_url = base_url,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)

sentence_transformer = OllamaSentenceTransformer()


In [4]:
db_path = "./chroma_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)
    print("기존 ChromaDB 삭제 완료")

In [5]:
from chromadb.utils import embedding_functions

bge_embed = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="BAAI/bge-m3"
)

In [9]:
# ChromaDB 클라이언트
client = chromadb.PersistentClient(path =db_path)

#collection 삭제
# client.delete_collection('medical_qa')

# Collection 생성
collection = client.get_or_create_collection('medical_qa',
                                            #  embedding_function=bge_embed
                                             )


In [10]:
collection.configuration

{'hnsw': {'space': 'l2',
  'ef_construction': 100,
  'ef_search': 100,
  'max_neighbors': 16,
  'resize_factor': 1.2,
  'sync_threshold': 1000},
 'spann': None,
 'embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x77170e71a9c0>}

In [18]:
question

'12세 소아가 반복적인 복통과 혈뇨로 내원하였다. 환자는 과거 낫적혈구병(SCD) 진단을 받은 적이 있다. 이 환자의 신장 기능 평가를 위해 가장 적절한 검사는 무엇인가?  \n1) 혈중 요산 농도  \n2) 사구체 여과율(GFR) 측정  \n3) 소변 단백/크레아티닌 비율  \n4) 신장 초음파  \n5) 혈중 크레아티닌'

In [12]:
path = "../../../../data/medical_data/"
for domain in os.listdir(path):
    print(domain.split('/')[-1])
    for file in tqdm(os.listdir(os.path.join(path, domain))[:3]):
        file_path = os.path.join(path, domain, file)
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            json_file = f.read()
            data = json.loads(json_file)

        print(data)
        question = data.get("question" , "").strip()
        
        # embedding = sentence_transformer.encode(question)
        # embedding = embedding.astype(float).tolist()
        # # print(embedding)
        # collection.add(
        #     ids = [file],
        #     documents=[question],
        #     embeddings = [embedding],
        #     metadatas = [{
        #         "answer" : data['answer'],
        #         "qa_id" : data["qa_id"],
        #         "domain" : data["domain"],
        #         "q_type" : data["q_type"],
        #     }],
        # )     
        # collection.update(
        #     ids = [file],
        #     # documents=[question],
        #     embeddings = [embedding],
            
        # )   

        # break
    # break
        
    

TL_내과


100%|██████████| 3/3 [00:00<00:00, 663.41it/s]


{'qa_id': 61200, 'domain': 17, 'q_type': 1, 'question': '62세 남성이 최근 발생한 심근경색 이후 심혈관 사건의 2차 예방을 위해 내원하였다. 환자는 이미 고혈압과 고콜레스테롤 혈증에 대해 최적의 약물 치료를 받고 있으며, 염증 생체지표(CRP)가 상승된 상태이다. 이 환자에게 추가로 고려할 수 있는 치료 옵션은 무엇인가?\n\n1) 고용량 아스피린  \n2) 고용량 베타 차단제  \n3) 저용량 콜히친  \n4) 칼슘 채널 차단제  \n5) 이뇨제', 'answer': '3) 저용량 콜히친'}
{'qa_id': 611190, 'domain': 17, 'q_type': 1, 'question': '45세 남성이 X-연관 심장 판막 이형성증으로 진단받았다. 이 질환과 관련된 유전적 돌연변이는 다음 중 어떤 세포 골격 단백질과 관련이 있는가?  \n1) 콜라겐  \n2) 엘라스틴  \n3) 필라민 A  \n4) 피브로넥틴  \n5) 케라틴', 'answer': '3) 필라민 A'}
{'qa_id': 624330, 'domain': 17, 'q_type': 3, 'question': '62세 여성 환자가 최근 3일간 좌측 흉부에 발생한 발진과 통증을 주소로 내원하였다. 환자는 고혈압과 고지혈증으로 치료 중이며, 면역억제제를 복용하고 있지 않다. 진찰 결과, 좌측 흉부에 군집성의 소수포가 관찰되었다. 이 환자의 가장 가능성 높은 진단과 적절한 치료 방안을 서술하시오.', 'answer': '이 환자는 좌측 흉부에 군집성 소수포와 통증이 동반된 증상을 보이고 있으며, 면역억제제를 복용하지 않는 상태에서 나타난 점을 고려할 때, 대상포진(herpes zoster)으로 진단할 수 있다. 치료로는 항바이러스제인 아시클로버(Acyclovir), 발라시클로버(Valacyclovir), 또는 팜시클로버(Famciclovir) 중 하나를 선택하여 투여하며, 발진 발생 후 72시간 이내에 치료를 시작하는 것이 중요하다. 또한, 환자의 통증

100%|██████████| 3/3 [00:00<00:00, 1092.27it/s]


{'qa_id': 6362, 'domain': 14, 'q_type': 1, 'question': '임신을 계획 중인 여성에게 권장되는 사전 건강 관리로 가장 적절한 것은?\n1) 엽산 섭취를 줄여 신경관 결손을 예방한다.\n2) 흡연, 음주, 약물 복용을 권장한다.\n3) 현재 체중을 유지하고 체중 변화는 피한다.\n4) 임신 전 당뇨병이나 고혈압과 같은 건강 문제를 해결한다.\n5) 백신 접종을 피한다.', 'answer': '4) 임신 전 당뇨병이나 고혈압과 같은 건강 문제를 해결한다.'}
{'qa_id': 611122, 'domain': 14, 'q_type': 1, 'question': 'HIV 감염 여성을 대상으로 한 자궁경부암 검진에서 HPV DNA 검사를 권장하는 주요 이유는 무엇인가?  \n1) 낮은 민감도  \n2) 낮은 특이도  \n3) 비용 효율성  \n4) 높은 민감도  \n5) 치료 과정 단축', 'answer': '4) 높은 민감도'}
{'qa_id': 627962, 'domain': 14, 'q_type': 1, 'question': '35세 여성이 최근 만성적인 질 분비물과 가려움증을 호소하며 병원을 방문하였다. 검사 결과, 그녀는 세균성 질염(Bacterial Vaginosis)으로 진단되었다. 담당 의사는 티니다졸(tinidazole)을 처방하기로 결정하였다. 티니다졸의 약물 작용 기전에 대한 설명으로 가장 적절한 것은 무엇인가?  \n1) 티니다졸은 세포막을 파괴하여 세균을 사멸시킨다.  \n2) 티니다졸은 단백질 합성을 억제하여 세균 성장을 저해한다.  \n3) 티니다졸은 엽산 합성을 저해하여 세균 증식을 억제한다.  \n4) 티니다졸은 DNA 손상을 유발하여 세균과 원생동물의 세포 사멸을 유도한다.  \n5) 티니다졸은 세포벽 합성을 억제하여 세균을 사멸시킨다.', 'answer': '4) 티니다졸은 DNA 손상을 유발하여 세균과 원생동물의 세포 사멸을 유도한다.'}
TL_응급의학과


100%|██████████| 3/3 [00:00<00:00, 1338.18it/s]


{'qa_id': 2476, 'domain': 16, 'q_type': 3, 'question': '고혈당성 혼수의 주요 원인과 치료 방법을 구체적으로 설명하시오.', 'answer': '고혈당성 혼수는 주로 고삼투성 비케톤성 혼수(Hyperosmolar Hyperglycemic State, HHS)와 당뇨병성 케톤산증(Diabetic Ketoacidosis, DKA)로 인해 발생한다. HHS는 주로 제2형 당뇨병 환자에서 발생하며, 심한 고혈당과 탈수로 인해 의식 저하가 나타난다. DKA는 주로 제1형 당뇨병 환자에서 발생하며, 인슐린 부족으로 인해 케톤체가 축적되어 대사성 산증을 유발한다. 치료는 빠른 수액 공급으로 탈수를 교정하고, 인슐린을 투여하여 혈당을 조절하며, 전해질 불균형을 교정하는 것이 포함된다. 또한, 감염이나 스트레스와 같은 유발 요인을 찾아 치료하는 것이 중요하다. 고혈당성 혼수는 생명을 위협할 수 있는 응급 상황이므로 신속하고 체계적인 치료가 필요하다.'}
{'qa_id': 620008, 'domain': 16, 'q_type': 1, 'question': '7세 남아가 응급실에 도착하여 인공호흡기 삽관이 필요하게 되었다. 의료진은 삽관 과정에서 발생할 수 있는 서맥(심박수 감소)을 예방하기 위해 약물 치료를 고려하고 있다. 다음 중 이 상황에서 서맥 예방을 위해 사용할 수 있는 약물은 무엇인가?  \n1) 아트로핀  \n2) 이부프로펜  \n3) 아세트아미노펜  \n4) 라니티딘  \n5) 에페드린', 'answer': '1) 아트로핀'}
{'qa_id': 65262, 'domain': 16, 'q_type': 1, 'question': '32세 남자가 정원에서 일을 하다가 갑자기 손에 통증과 함께 붉은 발진이 생겼다며 병원에 왔다. 환자는 이전에 벌에 쏘였을 때 경미한 알레르기 반응을 보인 적이 있다고 한다. 신체검사에서 손에 국소적인 붉은 발진과 부종이 관찰되었으며, 전신적인 증상은 없다. 이 환자에게 필요한 초기 처치는?  \n1

100%|██████████| 3/3 [00:00<00:00, 1114.22it/s]

{'qa_id': 1920, 'domain': 15, 'q_type': 2, 'question': '소아에서 반복적인 발작을 진단하기 위해 확인해야 하는 뇌파 검사 소견은?', 'answer': '비정상 방전'}
{'qa_id': 67405, 'domain': 15, 'q_type': 1, 'question': '미숙아의 산소 유리기 손상 방지를 위한 항산화제 보충에 대한 설명으로 옳은 것은?  \n1) 루테인과 제아잔틴은 체내에서 합성되지 않는다.  \n2) 베타카로틴은 루테인보다 미숙아에서 ROP 예방에 더 효과적이다.  \n3) 루테인과 제아잔틴은 주로 어두운 잎 채소에 함유되어 있다.  \n4) 루테인과 제아잔틴은 미숙아의 신경 발달에 부정적인 영향을 미친다.  \n5) 루테인과 제아잔틴 보충은 미숙아의 혈중 카로티노이드 농도를 낮춘다.', 'answer': '3) 루테인과 제아잔틴은 주로 어두운 잎 채소에 함유되어 있다.'}
{'qa_id': 612530, 'domain': 15, 'q_type': 1, 'question': '12세 소아가 반복적인 복통과 혈뇨로 내원하였다. 환자는 과거 낫적혈구병(SCD) 진단을 받은 적이 있다. 이 환자의 신장 기능 평가를 위해 가장 적절한 검사는 무엇인가?  \n1) 혈중 요산 농도  \n2) 사구체 여과율(GFR) 측정  \n3) 소변 단백/크레아티닌 비율  \n4) 신장 초음파  \n5) 혈중 크레아티닌', 'answer': '2) 사구체 여과율(GFR) 측정'}


In [17]:
all_doc = collection.get()

In [18]:
all_doc

{'ids': ['필수_61200.json',
  '필수_611190.json',
  '필수_624330.json',
  '필수_6362.json',
  '필수_611122.json',
  '필수_627962.json',
  '필수_2476.json',
  '필수_620008.json',
  '필수_65262.json',
  '필수_1920.json',
  '필수_67405.json',
  '필수_612530.json'],
 'embeddings': None,
 'documents': ['62세 남성이 최근 발생한 심근경색 이후 심혈관 사건의 2차 예방을 위해 내원하였다. 환자는 이미 고혈압과 고콜레스테롤 혈증에 대해 최적의 약물 치료를 받고 있으며, 염증 생체지표(CRP)가 상승된 상태이다. 이 환자에게 추가로 고려할 수 있는 치료 옵션은 무엇인가?\n\n1) 고용량 아스피린  \n2) 고용량 베타 차단제  \n3) 저용량 콜히친  \n4) 칼슘 채널 차단제  \n5) 이뇨제',
  '45세 남성이 X-연관 심장 판막 이형성증으로 진단받았다. 이 질환과 관련된 유전적 돌연변이는 다음 중 어떤 세포 골격 단백질과 관련이 있는가?  \n1) 콜라겐  \n2) 엘라스틴  \n3) 필라민 A  \n4) 피브로넥틴  \n5) 케라틴',
  '62세 여성 환자가 최근 3일간 좌측 흉부에 발생한 발진과 통증을 주소로 내원하였다. 환자는 고혈압과 고지혈증으로 치료 중이며, 면역억제제를 복용하고 있지 않다. 진찰 결과, 좌측 흉부에 군집성의 소수포가 관찰되었다. 이 환자의 가장 가능성 높은 진단과 적절한 치료 방안을 서술하시오.',
  '임신을 계획 중인 여성에게 권장되는 사전 건강 관리로 가장 적절한 것은?\n1) 엽산 섭취를 줄여 신경관 결손을 예방한다.\n2) 흡연, 음주, 약물 복용을 권장한다.\n3) 현재 체중을 유지하고 체중 변화는 피한다.\n4) 임신 전 당뇨병이나 고혈압과 같은 건강 문제를 해결한다.\n5) 백신 접종을 피한다.',
  'HIV 감염 여성을 대